In [2]:
import pandas as pd
import yfinance as yf

# List of S&P 500 tickers to work with as a sample
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

# Fetch stock data for the S&P 500 tickers
def fetch_stock_data(tickers):
    stock_info = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap')
        sector = info.get('sector')
        if market_cap and sector:
            stock_info.append({
                'Ticker': ticker,
                'Sector': sector,
                'MarketCap': market_cap
            })
    return pd.DataFrame(stock_info)

# Fetch data
stock_df = fetch_stock_data(sp500_tickers)

# Compute sector value counts and select the top 3 sectors
sector_counts = stock_df['Sector'].value_counts()
top_sectors = sector_counts.nlargest(5).index.tolist()
print("Top 5 sectors by value count:", top_sectors)

# Function to categorize market cap
def categorize_market_cap(market_cap):
    if market_cap >= 10e9:
        return 'Large-Cap'
    elif 2e9 <= market_cap < 10e9:
        return 'Mid-Cap'
    elif 300e6 <= market_cap < 2e9:
        return 'Small-Cap'
    else:
        return 'Micro-Cap'

# Categorize market caps
stock_df['MarketCapCategory'] = stock_df['MarketCap'].apply(categorize_market_cap)

# Filter for large-cap stocks
large_cap_stocks = stock_df[stock_df['MarketCapCategory'] == 'Large-Cap']
print("Large-cap stocks:\n", large_cap_stocks.head())

# Sort and select the top 5 large-cap stocks from each of the top 3 sectors
top_large_cap_stocks = []
for sector in top_sectors:
    sector_large_cap_stocks = large_cap_stocks[large_cap_stocks['Sector'] == sector]
    sector_large_cap_stocks = sector_large_cap_stocks.sort_values(by='MarketCap', ascending=False).head(5)
    print(f"Top 5 large-cap stocks for sector {sector}:\n", sector_large_cap_stocks[['Ticker', 'MarketCap']])
    top_large_cap_stocks.extend(sector_large_cap_stocks['Ticker'].tolist())

print("Top 5 large-cap stocks from each of the top 5 sectors:", top_large_cap_stocks)


Top 5 sectors by value count: ['Technology', 'Industrials', 'Healthcare', 'Financial Services', 'Consumer Cyclical']
Large-cap stocks:
   Ticker       Sector     MarketCap MarketCapCategory
0    MMM  Industrials   57073655808         Large-Cap
1    AOS  Industrials   12575485952         Large-Cap
2    ABT   Healthcare  179112296448         Large-Cap
3   ABBV   Healthcare  287713165312         Large-Cap
4    ACN   Technology  190907285504         Large-Cap
Top 5 large-cap stocks for sector Technology:
     Ticker      MarketCap
317   MSFT  3188758151168
39    AAPL  2949514264576
347   NVDA  2346333503488
74    AVGO   648418623488
356   ORCL   342546776064
Top 5 large-cap stocks for sector Industrials:
     Ticker     MarketCap
209     GE  176232218624
89     CAT  175604269056
458    UNP  143354265600
392    RTX  140170248192
160    ETN  134655639552
Top 5 large-cap stocks for sector Healthcare:
     Ticker     MarketCap
167    LLY  763336785920
462    UNH  481867563008
266    JNJ  36398